### Dependencies

In [4]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import os
from dotenv import load_dotenv

In [16]:
# from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [8]:
load_dotenv()

True

### Test LangChain OpenAi Integration

In [18]:
# Create an LLM instance
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Make a simple inference
user_prompt = """
What is the capital of the US

"""
messages = [ HumanMessage(content=user_prompt), 
             SystemMessage(content="Make sure to present your response in bullet point")
           ]

response = llm.invoke(messages)

print(response.content)

- The capital of the United States is Washington, D.C.


In [19]:
x = llm.invoke(messages)

### Data

In [21]:
df = pd.read_csv("Customer.csv")

df.head()

,customer_Id,DOB,Gender,city_code
0,268408,02-01-1970,M,4.0
1,269696,07-01-1970,F,8.0
2,268159,08-01-1970,F,8.0
3,270181,10-01-1970,F,2.0
4,268073,11-01-1970,M,1.0


### Customised Agents (Helping Functions)

In [24]:
def data_summary(df):
    """"
    Returns a summary of 
    """
    df = df.copy()
    return df.info().Column

In [25]:
data_summary()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5647 entries, 0 to 5646
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   customer_Id  5647 non-null   int64  
 1   DOB          5647 non-null   object 
 2   Gender       5645 non-null   object 
 3   city_code    5645 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 176.6+ KB


AttributeError: 'NoneType' object has no attribute 'Column'